In [1]:
import numpy as np
import pandas as pd

In [2]:
from env import *

In [3]:
from multiprocessing import Process, Pool
from multiprocessing.managers import Queue
import time

In [21]:
def f(x,y):
    return x+1

In [24]:
f(2,3,4)

TypeError: f() takes 2 positional arguments but 3 were given

In [7]:
def gogo(obj, gpus):
    print('ok')
    gpu_id = 0
    # gpu_id = gpus.get()
    np.random.seed()
    sleep = np.random.randint(1,6)
    print(f'begin {obj} {gpu_id} {sleep}')
    time.sleep(sleep)
    # gpus.put(gpu_id)
    print(f'end {obj} {gpu_id}')


In [8]:
gpus = Queue(2)
for i in range(2):
    gpus.put(i)

ps = Pool(4)
for c in ['a', 'b', 'c', 'd']:
    ps.apply_async(gogo, (c, 0,))
ps.close()
ps.join()

okokokok



begin c 0 5begin d 0 4begin b 0 2begin a 0 4



end b 0
end d 0end a 0

end c 0


In [31]:
class TrainerProcess(Process):

    def __init__(self, obj, gpus):
        super().__init__()
        self.obj = obj
        self.gpus = gpus
    
    def run(self):
        gpu_id = self.gpus.get()
        np.random.seed()
        sleep = np.random.randint(1,6)
        print(f'begin {self.obj} {gpu_id} {sleep}')
        time.sleep(sleep)
        self.gpus.put(gpu_id)
        print(f'end {self.obj} {gpu_id}')

gpus = Queue(2)
for i in range(2):
    gpus.put(i)

ps = []
for c in ['a', 'b', 'c', 'd']:
    p = TrainerProcess(c,gpus)
    ps.append(p)
for i in ps:
    i.start()

for i in ps:
    i.join()

        


begin a 0 2
begin b 1 1
end b 1
begin c 1 3
end a 0
begin d 0 1
end d 0
end c 1


In [27]:
np.random.randint(9,30)

12

In [4]:
def preprocess(tickpath, filename):
    tickdf = pd.read_csv(os.path.join(tickpath, filename),names=['date','time','ms','lastprice','volume','bid','bidv','ask','askv','opi','tur','contract'])
    tickdf[['lastprice','volume','bid','bidv','ask','askv','opi','tur']] = tickdf[['lastprice','volume','bid','bidv','ask','askv','opi','tur']].astype('float32')
    tickdf['datestr'] = tickdf['date'].apply(lambda x: '%s-%s-%s'%(str(x)[:4], str(x)[4:6], str(x)[6:8]))
    tickdf['timestr'] = tickdf['time'].apply(lambda x: '%s:%s:%s'%(format(x, '06d')[:-4], format(x, '06d')[-4:-2], format(x, '06d')[-2:]))
    tickdf['timestr'] = tickdf['timestr']+'.'+tickdf['ms'].apply(lambda x: format(x, '03d'))
    tickdf = tickdf.set_index(pd.to_datetime(tickdf['datestr']+' '+tickdf['timestr']))
    tickdf.index.name = 'datetime'
    # try:
    tickdf.loc[tickdf['bid']<0.9*tickdf['lastprice'], 'bid'] = np.nan
    tickdf.loc[tickdf['ask']<0.9*tickdf['lastprice'], 'ask'] = np.nan
    # except TypeError as e:
        
    #     print((tickdf['bid'][tickdf.bid.apply(lambda x:type(x) is str)]))
    #     print(float(tickdf['bid'].iloc[0]))
    #     print((tickdf['ask'][tickdf.bid.apply(lambda x:type(x) is str)]))
    #     print(float(tickdf['ask'].iloc[0]))
    #     raise e
    tickdf[['bid','ask']] = tickdf[['bid','ask']].fillna(method='ffill').fillna(tickdf.iloc[0]['lastprice'])
    tickdf['midprice'] = (tickdf['ask']+tickdf['bid'])/2.0

    tdata = tickdf[['bid','ask','bidv','askv','volume']].copy()
    std = tdata.iloc[0,0]
    tdata.loc[:,'bid'] = tdata['bid']-std
    tdata.loc[:,'ask'] = tdata['ask']-std

    def norm(x):
        return (x-x.mean())/x.std()

    tdata.loc[:,'askv'] = norm(tdata.askv)
    tdata.loc[:,'bidv'] = norm(tdata.bidv)
    tdata.loc[:,'volume'] = norm(tdata.volume)
    # print(type(tdata.isnull()))
    if pd.isnull(tdata).any().any():
        print('drop it')
        print(filename)
        raise ValueError
    return tdata

In [5]:
import os
cwd = '/Data/database/data_zltick/rb'
datas = []
for _,__, files in os.walk(cwd):
    for file in files:
        if int(file[:4])<2018 or int(file[:4])>2020:
            continue
        try:
            data = preprocess(cwd, file)
            datas.append(data)
        except (IndexError, ValueError):
            pass

In [5]:
for data in datas:
    if pd.isnull(data).any().any():
        print(34)

In [6]:
# import matplotlib.pyplot as plt
# fig, ax1 = plt.subplots()

# ax2 = ax1.twinx()
# ax1.plot(tdata.bid[500:800])
# ax2.plot(tdata.volume[500:800], color='red')

# plt.show()

In [8]:
tot = 0
env.seed(666)
for i in range(1024):
    done = False
    state = env.reset()
    action = 0
    while not done:
        state, reward, done, _ = env.step(action)
        if state[0] == 1:
            action = 1
    tot += reward
print(tot/1024)

-0.0576171875


In [9]:
env.seed(666)
tot = 0
for i in range(1024):
    done = False
    state = env.reset()
    while not done:
        action = env.action_space.sample()
        state, reward, done, _ = env.step(action)
    tot += reward
print(tot/1024)

-0.0244140625


In [15]:
import elegantrl.agent as agent
from elegantrl.config import Arguments
env_args = {
    'env_num': 1,
    'env_name': 'executioner',
    'max_step': 600,
    'state_dim': (100+1)*5,
    'action_dim': 2,
    'if_discrete': True,
    'target_return': 0.31
}

In [16]:
env = SellMarket(datas=datas,back_length=100,time_limit=600)
args = Arguments(agent.AgentD3QN,env=env, env_args=env_args)

In [17]:
np.isnan(env.datas).any()

False

In [18]:
from elegantrl.run import train_and_evaluate

In [19]:
args.target_step = args.max_step * 50
args.eval_times = 2 ** 10
args.net_dim = 512
args.batch_size = 1024
args.gamma = 1
args.explore_rate = 0.1
args.seed = 666
args.learning_rate = 2**-15
args.eval_gap = 2 ** 6
# args.break_step = 4e6
train_and_evaluate(args)

| Arguments Remove cwd: ./executioner_D3QN_0
################################################################################
ID     Step    maxR |    avgR   stdR   avgS  stdS |    expR   objC   etc.
0  3.00e+04    0.07 |
0  3.00e+04    0.07 |    0.07    1.7     24    35 |   -0.01   0.02  -0.03
0  1.80e+05    0.07 |    0.06    2.5     47    72 |    0.00   0.02   0.00
0  3.30e+05    0.16 |
0  3.30e+05    0.16 |    0.16    1.3     14    17 |    0.01   0.02  -0.13
0  5.10e+05    0.20 |
0  5.10e+05    0.20 |    0.20    1.0     11    15 |    0.01   0.03  -0.11
0  6.60e+05    0.25 |
0  6.60e+05    0.25 |    0.25    0.9      9    11 |    0.02   0.02   0.05
0  8.10e+05    0.26 |
0  8.10e+05    0.26 |    0.26    0.8      7     9 |    0.01   0.03  -0.00
0  9.60e+05    0.26 |    0.25    0.9      8    10 |    0.02   0.02   0.00
0  1.11e+06    0.27 |
0  1.11e+06    0.27 |    0.27    1.3     20    45 |    0.01   0.02   0.27
0  1.23e+06    0.27 |    0.19    2.2     41    95 |    0.02   0.02   0.22
0 

KeyboardInterrupt: 

In [21]:
# age = args.agent().save_or_load_agent(cwd='./executioner_D3QN_0', if_save=False)
from elegantrl.run import *

In [22]:
args.learner_gpus

0

In [38]:
import torch
torch.set_grad_enabled(False)
# args.init_before_training()
gpu_id = args.learner_gpus

'''init'''
env = build_env(args.env, args.env_func, args.env_args)

age = init_agent(args, gpu_id, env)

In [39]:
age.save_or_load_agent('./executioner_D3QN_0', if_save=False)

In [40]:
# for seed in range(100):
seed = 123456
env.seed(seed)
tot = 0
tmp = 0
device = torch.device(f'cuda:{args.learner_gpus}')
for i in range(100000):
    done = False
    state = env.reset()
    steps = 0
    # print(state.reshape((-1, 5)))
    while not done:
        steps += 1
        ten_s = torch.as_tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        ten_a = age.act(ten_s)
        # print(ten_a)
        ten_a = ten_a.argmax(dim=1).cpu().numpy()[0]
        state, reward, done, _ = env.step(ten_a)  # different
        # print(state[-5:])
        # print(reward)
    tot += reward
    tmp += reward
    if i % 1000 == 0:
        print(f'1000 times avg: {tmp/1000}')
        tmp = 0
    # env.episode_data[:300].to_csv('tmp.csv')
    # break
    # print(steps)
print(tot/100000)

1000 times avg: 0.001
1000 times avg: 0.305
1000 times avg: 0.266
1000 times avg: 0.308
1000 times avg: 0.215
1000 times avg: 0.266
1000 times avg: 0.292
1000 times avg: 0.36
1000 times avg: 0.314
1000 times avg: 0.234
1000 times avg: 0.322
1000 times avg: 0.289
1000 times avg: 0.288
1000 times avg: 0.316
1000 times avg: 0.279
1000 times avg: 0.304
1000 times avg: 0.271
1000 times avg: 0.28
1000 times avg: 0.295
1000 times avg: 0.181
1000 times avg: 0.28
1000 times avg: 0.137
1000 times avg: 0.27
1000 times avg: 0.259
1000 times avg: 0.266
1000 times avg: 0.28
1000 times avg: 0.254
1000 times avg: 0.301
1000 times avg: 0.295
1000 times avg: 0.229
1000 times avg: 0.214
1000 times avg: 0.311
1000 times avg: 0.269
1000 times avg: 0.266
1000 times avg: 0.27
1000 times avg: 0.146
1000 times avg: 0.292
1000 times avg: 0.323
1000 times avg: 0.32
1000 times avg: 0.288
1000 times avg: 0.252
1000 times avg: 0.283
1000 times avg: 0.123
1000 times avg: 0.266
1000 times avg: 0.222
1000 times avg: 0

In [18]:
# for seed in range(100):
seed = 123456
env.seed(seed)
tot = 0
tmp = 0
device = torch.device(f'cuda:{args.learner_gpus}')
for i in range(100000):
    done = False
    state = env.reset()
    steps = 0
    # print(state.reshape((-1, 5)))
    ten_a = 0
    while not done:
        steps += 1
        # ten_s = torch.as_tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        # ten_a = age.act(ten_s)
        # # print(ten_a)
        # ten_a = ten_a.argmax(dim=1).cpu().numpy()[0]
        state, reward, done, _ = env.step(ten_a)  # different
        if state[0] == 1:
            ten_a = 1

        # print(state[-5:])
        # print(reward)
    tot += reward
    tmp += reward
    if i % 1000 == 0:
        print(f'1000 times avg: {tmp/1000}')
        tmp = 0
    # env.episode_data[:300].to_csv('tmp.csv')
    # break
    # print(steps)
print(tot/100000)

1000 times avg: 0.001
1000 times avg: 0.163
1000 times avg: 0.285
1000 times avg: 0.167
1000 times avg: 0.09
1000 times avg: -0.044
1000 times avg: 0.201
1000 times avg: 0.195
1000 times avg: 0.181
1000 times avg: 0.07
1000 times avg: 0.167
1000 times avg: 0.084
1000 times avg: 0.257
1000 times avg: 0.271
1000 times avg: 0.278
1000 times avg: 0.191
1000 times avg: -0.068
1000 times avg: 0.116
1000 times avg: 0.201
1000 times avg: 0.394
1000 times avg: -0.016
1000 times avg: 0.037
1000 times avg: -0.105
1000 times avg: 0.124
1000 times avg: 0.193
1000 times avg: 0.185
1000 times avg: 0.12
1000 times avg: 0.266
1000 times avg: 0.163
1000 times avg: 0.016
1000 times avg: -0.01
1000 times avg: 0.227
1000 times avg: -0.133
1000 times avg: -0.135
1000 times avg: 0.041
1000 times avg: -0.04
1000 times avg: 0.145
1000 times avg: -0.099
1000 times avg: 0.154
1000 times avg: 0.112
1000 times avg: 0.289
1000 times avg: 0.148
1000 times avg: 0.104
1000 times avg: 0.352
1000 times avg: 0.014
1000 t

In [21]:
for seed in range(100):
    tot = 0
    env.seed(seed)
    for i in range(1000):
        done = False
        state = env.reset()
        action = 0
        while not done:
            state, reward, done, _ = env.step(action)
            if state[0] == 1:
                action = 1
        tot += reward
    print(tot/1000)

0.193
-0.09
-0.106


KeyboardInterrupt: 

In [38]:
tot = 0
env.seed(1)
for i in range(1000):
    done = False
    state = env.reset()
    action = 1
    while not done:
        state, reward, done, _ = env.step(action)
    tot += reward
print(tot/1000)

-0.046


In [41]:
tot = 0
env.seed(1)
for i in range(10000):
    done = False
    state = env.reset()
    while not done:
        action = env.action_space.sample()
        state, reward, done, _ = env.step(action)
    tot += reward
print(tot/10000)

-0.0302


In [28]:
tmp = np.load(r'executioner_D3QN_0\replay_0.npz')

In [40]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('tkagg')
plt.plot(data['bid'])
plt.show()

In [35]:
tmp['buf_other']

array([[ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ],
       ...,
       [ 0.  ,  0.99,  2.  ],
       [ 0.  ,  0.99,  2.  ],
       [-0.  ,  0.  ,  2.  ]], dtype=float16)

In [5]:
a = [(1,2,3),(3,4,5),(5,6,5),(7,8,5)]
b = list(map(list, zip(*a)))

In [7]:
import torch
b

[[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 5, 5]]